In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load the image
image_path = "base_image_1.jpg"
image = Image.open(image_path)
track_value = 165

# Read coordinates from the CSV file
df = pd.read_csv("Trkwise_Framewise.csv")
df_track = df[(df['Track'] == track_value)]  # Filter for Track == 14 and first 1000 frames

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(image)

# Initialize lists to store angles and coordinates
angles = []
resultant_vectors = []
coordinates = list(zip(df_track['xc'], df_track['yc']))

def get_direction(angle):
    
    angle = angle % 360
    
    if angle < -22.5:
        angle += 360 
    elif angle >= 337.5:
        angle -= 360 
        
    direction_map = {
        (-22.5, 22.5): 'E',
        (22.5, 67.5): 'NE',
        (67.5, 112.5): 'N',
        (112.5, 157.5): 'NW',
        (157.5, 202.5): 'W',
        (202.5, 247.5): 'SW',
        (247.5, 292.5): 'S',
        (292.5, 337.5): 'SE'
    }

    for key, value in direction_map.items():
        if key[0] <= angle < key[1]:
            return value
        
# Calculate angles between points using a window size of 5
window_size = 5
data=[]
for i in range(0, len(coordinates) - window_size + 1, window_size):
    p0 = np.array(coordinates[i])
    p1 = np.array(coordinates[i + window_size - 1])
    # Calculate vector
    v0 = p1 - p0
    v0[1] = -v0[1]
    
    angle = np.degrees(np.arctan2(v0[1],v0[0]))
    resultant_vectors.append(v0)
    direction = get_direction(angle)
    velocity = df_track['Velocity(kmph)'].iloc[i]
    
    data.append([track_value,(i/5)+1,p0,p1,velocity,angle,direction])
    
df_data = pd.DataFrame(data, columns=['Track', 'Srno','Initial Position [x y]','Final Position[x y]','Velocity', 'Angle', 'Direction'])

csv_filename = f"resultant_vector/resultant_vectors_track_{track_value}.csv"
df_data.to_csv(csv_filename, index=False)
    
# Plot resultant vectors on the image
for vector, (x, y) in zip(resultant_vectors, coordinates[::window_size]):
    ax.quiver(x, y, vector[0], vector[1], angles='xy', scale_units='xy', scale=1, color='red')



# Show the image with resultant vectors
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Resultant Vectors')
plt.show()
